In [1]:
from selenium import webdriver
import json
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
def web_scrape(url):
    options = Options()
    options.add_argument("--enable-performance-logging")
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    log = driver.get_log("performance")
    driver.quit()
    
    return log

In [4]:
url = "https://www.oddsportal.com/football/belgium/jupiler-pro-league/"
logs = web_scrape(url)

In [4]:
with open("oddsportal_network_dump.txt", 'w') as f:
    for i in logs:
        test = i['message']
        json_obj = json.loads(test)
        json_formatted = json.dumps(json_obj, indent=2)
        f.write(json_formatted)
    f.close()

In [5]:
with open("oddsportal_network_dump.json", 'w', encoding="utf-8") as f:
    f.write("[")
    for log in logs:
        network_log = json.loads(log["message"])["message"]
        if("Network.response" in network_log["method"] or "Network.request" in network_log["method"] or "Network.webSocket" in network_log["method"]):
            f.write(json.dumps(network_log)+",")
    f.write("{}]")

In [16]:
with open("oddsportal_network_dump.json", "r", encoding="utf8") as f:
    logs = json.loads(f.read())

urls = []
for log in logs:
    try:
        url = log["params"]["request"]["url"]
        urls.append(url)
    except:
        pass

filter_list = [".svg", ".png", "/js/", ".js", ".css", ".ico"]
filtered_urls = [url for url in urls if not any(filter_str in url for filter_str in filter_list)]

for url in filtered_urls:
    print(url)


https://www.oddsportal.com/football/belgium/jupiler-pro-league/
https://www.oddsportal.com/ajax-user-data/t/71596/?171455841533686c2d8930be81c843ffb7d4312605
https://www.oddsportal.com/res/public/fonts/OpenSans-Regular.woff2
https://www.oddsportal.com/res/public/fonts/OpenSans-Bold.woff2
https://content.livesportmedia.eu/delivery?zones=3991,3989,3990,5989,6061&clientType=PC&no_flash=1&programmatic=false
https://geolocation.onetrust.com/cookieconsentpub/v1/geo/location
https://www.oddsportal.com/ajax-getCount/MyBookmarks/
https://www.oddsportal.com/ajax-getCount/MyCoupon/
https://www.oddsportal.com/ajax-sport-country-tournament_/1/zun4qzgk/X0/1/?_=1714558415
https://www.googletagmanager.com/gtag/js?id=G-5YY4JY41P1&l=dataLayer&cx=c
https://region1.google-analytics.com/g/collect?v=2&tid=G-5YY4JY41P1&gtm=45je44t0v884757901z872785439za200&_p=1714558416820&gcs=G100&gcd=13p3pPp2p5&npa=1&dma_cps=-&dma=1&tcfd=10001&gdid=dYWJhMj&cid=1759878428.1714558417&ul=en-us&sr=1920x1080&uaa=x86&uab=64&uafv

# Finding the request(s) needed